In [1]:
# Main Libraries
import pandas as pd 
import numpy as np
from numpy import *

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

from multiprocessing import Pool
from itertools import product
import knntsp
import warnings
import importlib
warnings.filterwarnings('ignore')

NUM_CORES = 12
PREPROCESSED_PATH = './../preprocessed/'

In [2]:
# Reading groupped data
def read_groupped_data(filename):
    timeSeries = pd.read_csv(filename,index_col=['Unnamed: 0'], parse_dates=['Unnamed: 0'])
    timeSeries.rename(columns=lambda x: eval(x), inplace=True)
    timeSeries.index = pd.to_datetime(timeSeries.index)
    return timeSeries

In [3]:
# Reading time series data
timeSeries = read_groupped_data(PREPROCESSED_PATH + 'timeseries.csv')

In [4]:
# Removing time series without data
timeSeries = timeSeries.dropna(axis=1)

In [5]:
W_TO_TEST = [3,4,5] #, 27
K_TO_TEST = [1,2,3,4,5]
RANDOM_STATE = 1
TRAIN_SIZE = 0.86
FOR_TEST_PREDICTION = 20

In [7]:
importlib.reload(knntsp)

FRAC = 0.1

results = []
X = timeSeries.iloc[:-FOR_TEST_PREDICTION]#timeSeries.T.sample(frac=FRAC, replace=False, random_state=RANDOM_STATE).T.copy().iloc[:-FOR_TEST_PREDICTION]

to_run = []
for w in W_TO_TEST:
    for k in K_TO_TEST:

        for column in X.columns:
            to_run.append([X, column, w, k, TRAIN_SIZE])
            
with Pool(NUM_CORES) as pool:
    results = pool.starmap(knntsp.predict_with_knn, to_run)

In [19]:
SAVE_SIMPLE_W_K_SELECTION = False

simple_w_k_result = pd.DataFrame()
for result in results:
    current_X, current_Y, mean_error, w, k, cluster_label = result
    simple_w_k_result = simple_w_k_result.append(pd.Series([w, k, mean_error[0]]), ignore_index=True)

simple_w_k_result.columns = ['w','k', 'mean_error']
# groupped_mean = simple_w_k_result.groupby(['w','k'])['mean_error'].mean()
# groupped_mean
simple_w_k_result
# if SAVE_SIMPLE_W_K_SELECTION:
#     pd.DataFrame([[groupped_mean.idxmin()[0], groupped_mean.idxmin()[1], groupped_mean.loc[groupped_mean.idxmin()]]], columns=['w', 'k', 'mean_error']).to_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv')

# pd.DataFrame([[groupped_mean.idxmin()[0], groupped_mean.idxmin()[1], groupped_mean.loc[groupped_mean.idxmin()]]], columns=['w', 'k', 'mean_error'])

,w,k,mean_error
0,3.0,1.0,0.335562
1,3.0,1.0,0.970846
2,3.0,1.0,0.202694
3,3.0,1.0,1.214115
4,3.0,1.0,0.790645
5,3.0,1.0,0.295915
6,3.0,1.0,0.214274
7,3.0,1.0,1.239033
8,3.0,1.0,0.888700
9,3.0,1.0,1.262911


In [21]:
indexes = pd.Series([value*2660 for value in range(0, 16)])
simple_final_result = pd.DataFrame()
i = 0
for i in range(0, 2660):
    simple_final_result = simple_final_result.append(simple_w_k_result.loc[simple_w_k_result.loc[indexes]['mean_error'].idxmin()])
    indexes += 1
simple_final_result.reset_index(inplace=True)
simple_final_result.shape

(2660, 4)

In [22]:
SAVE_SIMPLE_W_K_SELECTION = True

if SAVE_SIMPLE_W_K_SELECTION:
    simple_final_result.to_csv(PREPROCESSED_PATH + 'simple_knntsp_w_k.csv')


In [8]:
importlib.reload(knntsp)
                 
cluster_labels = pd.read_csv(PREPROCESSED_PATH + 'standardization_based_cluster_labels.csv', header=-1, index_col=[0])
cluster_labels.columns = ['label']
cluster_labels.head()

CLUSTER_FRAC = 0.2

to_run = []
clustered_results = []
for name, group in cluster_labels.groupby(['label']):    
    group_sample = group.sample(frac=CLUSTER_FRAC, replace=False, random_state=RANDOM_STATE).copy()
    cluster_X = timeSeries.T.iloc[group_sample.index].T.iloc[:FOR_TEST_PREDICTION]    
    for w in W_TO_TEST:
        for k in K_TO_TEST:
            for column in cluster_X.columns:#[(16,21)]
                to_run.append([cluster_X, column, w, k, TRAIN_SIZE, False, name])

with Pool(NUM_CORES) as pool:
    clustered_results = pool.starmap(knntsp.predict_with_knn, to_run)

In [9]:
SAVE_CLUSTER_W_K_SELECTION = True

cluster_w_k_result = pd.DataFrame()
for result in clustered_results:
    current_X, current_Y, mean_error, w, k, cluster_label = result
    cluster_w_k_result = cluster_w_k_result.append(pd.Series([cluster_label, w, k, mean_error[0]]), ignore_index=True)

cluster_w_k_result.columns = ['cluster_label', 'w','k', 'mean_error']
cluster_groupped_mean = cluster_w_k_result.groupby(['cluster_label', 'w','k'], as_index=False)['mean_error'].mean()
cluster_groupped_mean = cluster_groupped_mean.iloc[cluster_groupped_mean.groupby(['cluster_label'])['mean_error'].idxmin().values]
cluster_groupped_mean.sort_values('cluster_label', inplace=True)
cluster_groupped_mean.reset_index(drop=True, inplace=True)

if SAVE_CLUSTER_W_K_SELECTION:
    cluster_groupped_mean.to_csv(PREPROCESSED_PATH + 'cluster_knntsp_w_k.csv')

cluster_groupped_mean.shape

(90, 4)

In [10]:
cluster_groupped_mean

,cluster_label,w,k,mean_error
0,1.0,3.0,2.0,0.088561
1,2.0,4.0,5.0,0.797070
2,3.0,4.0,2.0,0.593919
3,4.0,3.0,3.0,1.075806
4,5.0,3.0,3.0,0.923588
5,6.0,3.0,4.0,0.307258
6,7.0,4.0,3.0,1.056147
7,8.0,5.0,1.0,0.428838
8,9.0,5.0,5.0,1.050773
9,10.0,4.0,5.0,0.652148
